In [1]:
# Встановлення необхідних бібліотек
!pip install nltk ipywidgets

# Імпорт бібліотек
import re
import random
from collections import Counter
import numpy as np
import nltk
import ipywidgets as widgets
from IPython.display import display, clear_output

# Завантаження ресурсів NLTK
nltk.download('punkt')

print("Середовище готове.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.1 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Середовище готове.


In [2]:
# Завантаження файлу корпусу з URL
!wget https://lang.org.ua/static/downloads/corpora/wiki_dump.tokenized.txt.bz2

# Розархівація файлу
# -d: decompress (розархівувати)
# -k: keep (зберегти оригінальний архів)
!bzip2 -dk wiki_dump.tokenized.txt.bz2

print("Корпус завантажено та розархівовано.")

--2025-06-27 15:26:29--  https://lang.org.ua/static/downloads/corpora/wiki_dump.tokenized.txt.bz2
Resolving lang.org.ua (lang.org.ua)... 65.21.91.242
Connecting to lang.org.ua (lang.org.ua)|65.21.91.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 422072383 (403M) [application/octet-stream]
Saving to: ‘wiki_dump.tokenized.txt.bz2’

wiki_dump.tokenized 100%[===================>] 402.52M  28.4MB/s    in 15s     

2025-06-27 15:26:44 (26.4 MB/s) - ‘wiki_dump.tokenized.txt.bz2’ saved [422072383/422072383]

Корпус завантажено та розархівовано.


In [4]:
import re
import random
from collections import Counter
import numpy as np
import nltk

# --- Допоміжні функції ---

def count_words(tokenized_sentences):
    """
    Підраховує частоту кожного слова в корпусі.
    """
    word_counts = {}
    for sentence in tokenized_sentences:
        for token in sentence:
            if token not in word_counts:
                word_counts[token] = 0
            word_counts[token] += 1
    return word_counts

def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
    """
    Отримує слова, які з'являються не менше count_threshold разів.
    """
    closed_vocab = []
    word_counts = count_words(tokenized_sentences)
    for word, cnt in word_counts.items():
        if cnt >= count_threshold:
            closed_vocab.append(word)
    return closed_vocab

def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<unk>"):
    """
    Замінює слова, які не входять до словника, на токен unknown_token.
    """
    vocabulary = set(vocabulary)
    replaced_tokenized_sentences = []
    for sentence in tokenized_sentences:
        replaced_sentence = []
        for token in sentence:
            if token in vocabulary:
                replaced_sentence.append(token)
            else:
                replaced_sentence.append(unknown_token)
        replaced_tokenized_sentences.append(replaced_sentence)
    return replaced_tokenized_sentences

def preprocess_data(train_data, test_data, count_threshold):
    """
    Виконує повну попередню обробку даних.
    """
    vocabulary = get_words_with_nplus_frequency(train_data, count_threshold)
    train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary)
    test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary)
    return train_data_replaced, test_data_replaced, vocabulary

# --- Оптимізована функція для читання великого файлу ---

def load_and_process_partially(filepath, num_lines_to_process):
    """
    Читає та обробляє лише вказану кількість рядків з великого файлу.
    """
    tokenized_sentences = []
    print(f"Починаємо читати та обробляти перші {num_lines_to_process} рядків з файлу...")

    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i >= num_lines_to_process:
                    break

                sentence = line.lower().strip()
                tokens = sentence.split()
                if tokens:
                    tokenized_sentences.append(tokens)

                if (i + 1) % 100000 == 0:
                    print(f"  Оброблено {i + 1} рядків...")

    except FileNotFoundError:
        print(f"Помилка: файл '{filepath}' не знайдено.")
        return []

    print(f"Завершено. Оброблено {len(tokenized_sentences)} речень.")
    return tokenized_sentences

# --- Основна частина запуску обробки ---

LINES_TO_PROCESS = 500000
corpus_path = 'wiki_dump.tokenized.txt'

tokenized_data = load_and_process_partially(corpus_path, LINES_TO_PROCESS)

if tokenized_data:
    random.seed(8)
    random.shuffle(tokenized_data)
    train_size = int(len(tokenized_data) * 0.8)
    train_data = tokenized_data[:train_size]
    test_data = tokenized_data[train_size:]

    print(f"\nРозмір тренувальної вибірки: {len(train_data)} речень")
    print(f"Розмір тестової вибірки: {len(test_data)} речень")

    print("\nПопередня обробка та створення словника...")
    count_threshold = 2
    train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, test_data, count_threshold)

    print(f"Розмір словника: {len(vocabulary)} слів")
    print("\nКрок 1 успішно завершено!")
else:
    print("Не вдалося обробити дані.")

Починаємо читати та обробляти перші 500000 рядків з файлу...
  Оброблено 100000 рядків...
  Оброблено 200000 рядків...
  Оброблено 300000 рядків...
  Оброблено 400000 рядків...
  Оброблено 500000 рядків...
Завершено. Оброблено 500000 речень.

Розмір тренувальної вибірки: 400000 речень
Розмір тестової вибірки: 100000 речень

Попередня обробка та створення словника...
Розмір словника: 189383 слів

Крок 1 успішно завершено!


In [6]:
def count_n_grams(data, n, start_token='<s>', end_token='</s>'):
    """Підраховує кількість n-грам у даних."""
    n_grams = {}
    for sentence in data:
        sentence = [start_token] * (n - 1) + sentence + [end_token]
        sentence = tuple(sentence)
        for i in range(len(sentence) - n + 1):
            n_gram = sentence[i:i+n]
            if n_gram in n_grams:
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1
    return n_grams

# Обчислюємо N-грами різних розмірів
print("Обчислення N-грам...")
n_gram_counts_list = []
for n in range(1, 6): # Від уніграм (n=1) до 5-грам (n=5)
    print(f"Побудова {n}-грам...")
    n_grams = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_grams)

print("N-грамні моделі успішно побудовано.")

Обчислення N-грам...
Побудова 1-грам...
Побудова 2-грам...
Побудова 3-грам...
Побудова 4-грам...
Побудова 5-грам...
N-грамні моделі успішно побудовано.


In [8]:
def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    """Оцінює ймовірність слова `word` з K-згладжуванням."""
    previous_n_gram = tuple(previous_n_gram)
    previous_n_gram_count = n_gram_counts.get(previous_n_gram, 0)
    denominator = previous_n_gram_count + k * vocabulary_size
    n_plus1_gram = previous_n_gram + (word,)
    n_plus1_gram_count = n_plus1_gram_counts.get(n_plus1_gram, 0)
    numerator = n_plus1_gram_count + k
    probability = numerator / denominator
    return probability

def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    """Оцінює ймовірності для всіх слів у словнику."""
    previous_n_gram = tuple(previous_n_gram)
    vocabulary = vocabulary + ["<unk>", "</s>"]
    vocabulary_size = len(vocabulary)
    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k)
        probabilities[word] = probability
    return probabilities

def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    """Обчислює перплексію для речення."""
    n = len(list(n_gram_counts.keys())[0])
    sentence = ["<s>"] * n + sentence + ["</s>"]
    sentence = tuple(sentence)
    N = len(sentence)
    product_pi = 1.0
    for t in range(n, N):
        n_gram = sentence[t-n:t]
        word = sentence[t]
        probability = estimate_probability(word, n_gram, n_gram_counts, n_plus1_gram_counts, len(vocabulary), k)
        product_pi *= (1 / probability)
    perplexity = product_pi**(1/float(N))
    return perplexity

# Оцінимо перплексію нашої біграмної моделі (n=2) на одному реченні з тестового набору
print("\nОцінка перплексії...")
example_sentence = test_data_processed[0]
perplexity = calculate_perplexity(example_sentence, n_gram_counts_list[0], n_gram_counts_list[1], len(vocabulary))
print(f"Речення: {' '.join(example_sentence)}")
print(f"Перплексія для біграмної моделі: {perplexity:.4f}")


Оцінка перплексії...
Речення: isbn <unk>
Перплексія для біграмної моделі: 29.7745


In [9]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    """Отримує список найімовірніших наступних слів."""
    model_suggestions = {}
    n_max = len(n_gram_counts_list) # Максимальний розмір N-грами

    for n in range(2, n_max + 1): # Починаємо з біграм (n=2)
        n_gram_counts = n_gram_counts_list[n-2]
        n_plus1_gram_counts = n_gram_counts_list[n-1]

        # Визначаємо контекст
        if len(previous_tokens) >= n - 1:
            previous_n_gram = previous_tokens[-(n-1):]
        else:
            # Доповнюємо токенами початку, якщо контекст занадто короткий
            num_padding = (n - 1) - len(previous_tokens)
            previous_n_gram = ["<s>"] * num_padding + previous_tokens

        probabilities = estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k)

        # Фільтруємо за початком слова
        for word, prob in probabilities.items():
            if start_with and not word.startswith(start_with):
                continue
            # Зберігаємо найвищу ймовірність для кожного слова
            if word not in model_suggestions or prob > model_suggestions[word]:
                model_suggestions[word] = prob

    # Сортуємо слова за їхньою ймовірністю
    sorted_suggestions = sorted(model_suggestions.items(), key=lambda x: x[1], reverse=True)

    return sorted_suggestions

# Приклад використання
previous_text = "я думаю що"
tokens = previous_text.split()
suggestions = get_suggestions(tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"\nПропозиції для тексту '{previous_text}':")
for word, prob in suggestions[:5]:
    print(f"- {word} (ймовірність: {prob:.6f})")


Пропозиції для тексту 'я думаю що':
- <unk> (ймовірність: 0.003208)
- в (ймовірність: 0.002967)
- не (ймовірність: 0.002645)
- він (ймовірність: 0.001895)
- є (ймовірність: 0.001600)


In [11]:
def run_interactive_console(n_gram_counts_list, vocabulary):
    """
    Запускає простий текстовий інтерфейс для тестування автозавершення.
    """
    print("\n--- Інтерактивний режим автозавершення ---")
    print("Введіть текст. Для виходу напишіть 'вихід' або 'exit'.")

    while True:
        # Отримуємо ввід від користувача
        text = input("\nВаш текст: ")

        # Умова для виходу з циклу
        if text.lower() in ['вихід', 'exit']:
            print("Завершення роботи.")
            break

        if not text:
            continue

        tokens = text.lower().strip().split()

        # Отримуємо пропозиції
        # Передаємо відфільтрований список N-грам, щоб уникнути помилок
        suggestions = get_suggestions(tokens, n_gram_counts_list, vocabulary, k=1.0)

        # Фільтруємо спец-токени ПЕРЕД виводом
        final_suggestions = [s for s in suggestions if s[0] not in ['<unk>', '<s>', '</s>']]

        if not final_suggestions:
            print("Немає пропозицій для даного контексту.")
        else:
            print("Найкращі пропозиції:")
            for word, prob in final_suggestions[:5]:
                print(f"- {word} (ймовірність: {prob:.6f})")

# --- Приклад запуску ---
# Після того, як ви навчили моделі (після Кроку 2), запустіть цю функцію.

# run_interactive_console(n_gram_counts_list, vocabulary)

In [12]:
 run_interactive_console(n_gram_counts_list, vocabulary)


--- Інтерактивний режим автозавершення ---
Введіть текст. Для виходу напишіть 'вихід' або 'exit'.

Ваш текст: Сьогодні хороший день щоб почати
Найкращі пропозиції:
- його (ймовірність: 0.000026)
- будівництво (ймовірність: 0.000021)
- пошук (ймовірність: 0.000016)
- переговори (ймовірність: 0.000016)
- свій (ймовірність: 0.000011)

Ваш текст: Хороший день щоб почати переговори
Найкращі пропозиції:
- з (ймовірність: 0.000380)
- про (ймовірність: 0.000111)
- між (ймовірність: 0.000047)
- в (ймовірність: 0.000042)
- щодо (ймовірність: 0.000037)

Ваш текст: день щоб почати переговори з
Найкращі пропозиції:
- них (ймовірність: 0.005902)
- яких (ймовірність: 0.003495)
- метою (ймовірність: 0.002770)
- іншими (ймовірність: 0.001529)
- боку (ймовірність: 0.001434)

Ваш текст: щоб почати переговори з іншими
Найкращі пропозиції:
- даними (ймовірність: 0.000441)
- словами (ймовірність: 0.000252)
- мовами (ймовірність: 0.000231)
- державами (ймовірність: 0.000136)
- країнами (ймовірність: 0.00012

KeyboardInterrupt: Interrupted by user